### LangGraph - Persistence, State Management & Real-World Applications

**Building on LangGraph Building Blocks**

This notebook continues from `langGraph_buildingblocks.ipynb` and covers:
- Persistence with checkpointers (save and resume execution)
- Thread-based state management (conversation memory)
- Cross-thread memory with stores
- Human-in-the-loop workflows with interrupts
- State inspection and time travel
- Production-ready patterns

**Prerequisites**: Complete the building blocks notebook first to understand State, Nodes, Edges, and Command.

**Key Concepts**:
- **Checkpointers**: Save graph state at each step for resumption
- **Threads**: Unique conversation IDs that track state history
- **Stores**: Cross-thread memory for user preferences/data
- **Interrupts**: Pause execution for human input/approval

We'll use Groq for LLMs and build practical chatbot examples.

In [53]:
# Core imports for advanced LangGraph patterns

from typing import TypedDict, Annotated, Literal
from typing_extensions import TypedDict as ExtTypedDict
from operator import add
import uuid

# LangGraph core
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.graph.message import add_messages
from langgraph.types import Command, interrupt

# Checkpointing and storage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.memory import InMemoryStore

# LangChain
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables import RunnableConfig
from langgraph.store.base import BaseStore

import os
from dotenv import load_dotenv

print("Imports successful")

Imports successful


In [54]:
from pathlib import Path

# Load .env from project root even if the kernel starts in a subfolder
env_candidates = [Path.cwd(), Path.cwd().parent]
env_path = next((p / ".env" for p in env_candidates if (p / ".env").exists()), None)
if not env_path:
    raise FileNotFoundError("Could not find .env next to the project root")
load_dotenv(env_path, override=True)

groq_key = os.getenv("GROQ_API_KEY")
if not groq_key:
    raise RuntimeError("GROQ_API_KEY missing; check your .env")
os.environ["GROQ_API_KEY"] = groq_key.strip()

from langchain_groq import ChatGroq

llm = ChatGroq(
    model="openai/gpt-oss-120b",
    temperature=0.1,
)

response = llm.invoke("Say 'Hello, LangChain!' in a creative way.")
print(response.content)

🌟✨ *A ripple of code whispers through the digital ether...* ✨🌟  

**“Hey there, LangChain! 🚀”**  

— a friendly ping from the world of prompts, ready to link ideas together!


## Part 1: Understanding Persistence - Why Checkpointers Matter

**The Problem Without Persistence**:
- Graphs forget everything after execution
- Can't pause and resume
- No conversation history
- Can't handle failures gracefully

**What Checkpointers Provide**:
1. **State snapshots**: Save state at each graph step
2. **Resumption**: Pick up where you left off
3. **Memory**: Maintain conversation context
4. **Human-in-the-loop**: Pause for approval/input
5. **Time travel**: Inspect and replay any step

**Key Concept - Threads**:
- Each conversation gets a unique `thread_id`
- All checkpoints for that conversation are stored in that thread
- Different threads = different conversations with separate histories

## Part 2: Simple Persistence Example

Let's build a basic chatbot that remembers conversation history.

In [55]:
# Define state for our chatbot
# Using MessagesState - a built-in that includes messages with add_messages reducer

class ChatbotState(MessagesState):
    # MessagesState already includes: messages: Annotated[list, add_messages]
    # We can add additional fields
    user_name: str  # Track user's name

print("State defined")

State defined


In [56]:
# Define chatbot node
# This node has access to full conversation history through state

def chatbot_node(state: ChatbotState) -> dict:
    """
    Simple chatbot that responds to user messages.
    Has access to full message history via state['messages'].
    """
    # Get conversation history
    messages = state['messages']
    
    # Add system message if user name is known
    if state.get('user_name'):
        system_msg = SystemMessage(content=f"You are a helpful assistant. The user's name is {state['user_name']}.")
        messages = [system_msg] + messages
    
    # Generate response using full conversation context
    response = llm.invoke(messages)
    
    # Return the response (add_messages will append it)
    return {"messages": [response]}

print("Chatbot node defined")

Chatbot node defined


In [57]:
# Build graph WITH checkpointer
# This is the key difference - we add a checkpointer

# Create checkpointer - saves state in memory
checkpointer = MemorySaver()

# Build graph
workflow = StateGraph(ChatbotState)
workflow.add_node("chatbot", chatbot_node)
workflow.add_edge(START, "chatbot")
workflow.add_edge("chatbot", END)

# Compile WITH checkpointer - this enables persistence
simple_chatbot = workflow.compile(checkpointer=checkpointer)

print("Graph compiled with checkpointer")

Graph compiled with checkpointer


## Testing the Persistent Chatbot

Notice how we pass `thread_id` in the config - this is REQUIRED when using checkpointers.

In [58]:
# Start a conversation
# The thread_id identifies this specific conversation

config = {"configurable": {"thread_id": "conversation_1"}}

# First message
result1 = simple_chatbot.invoke(
    {
        "messages": [HumanMessage(content="Hi, my name is Alice")],
        "user_name": ""
    },
    config=config
)

print("=== First Exchange ===")
for msg in result1["messages"]:
    print(f"{msg.__class__.__name__}: {msg.content}")

=== First Exchange ===
HumanMessage: Hi, my name is Alice
AIMessage: Hello Alice! Nice to meet you. How can I assist you today?


In [59]:
# Continue the SAME conversation
# Notice: We only pass the new message, not the full history
# The checkpointer automatically loads previous messages

result2 = simple_chatbot.invoke(
    {
        "messages": [HumanMessage(content="What's my name?")]
    },
    config=config  # Same thread_id
)

print("\n=== Follow-up (Should remember name) ===")
print(f"AI: {result2['messages'][-1].content}")


=== Follow-up (Should remember name) ===
AI: Your name is Alice.


In [60]:
# Start a DIFFERENT conversation (different thread)
# This is a completely separate conversation with no shared history

new_config = {"configurable": {"thread_id": "conversation_2"}}

result3 = simple_chatbot.invoke(
    {
        "messages": [HumanMessage(content="What's my name?")]
    },
    config=new_config  # Different thread_id
)

print("\n=== New Thread (Should NOT know name) ===")
print(f"AI: {result3['messages'][-1].content}")


=== New Thread (Should NOT know name) ===
AI: I don’t have any information about your name. Could you let me know what you’d like to be called?


## Part 3: State Inspection - Looking Inside the Checkpoint

Checkpointers save the complete state at each step.
We can inspect this state at any time.

In [61]:
# Get the current state of a thread
# This returns a StateSnapshot object

config = {"configurable": {"thread_id": "conversation_1"}}
current_state = simple_chatbot.get_state(config)

print("=== Current State Inspection ===")
print(f"\nState values: {current_state.values}")
print(f"\nNext nodes to execute: {current_state.next}")
print(f"\nCheckpoint ID: {current_state.config['configurable']['checkpoint_id']}")
print(f"\nNumber of messages: {len(current_state.values['messages'])}")

=== Current State Inspection ===

State values: {'messages': [HumanMessage(content='Hi, my name is Alice', additional_kwargs={}, response_metadata={}, id='3273c308-a742-4a8c-a6d5-6271651a1a81'), AIMessage(content='Hello Alice! Nice to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'We need to respond. The user says "Hi, my name is Alice". We should greet them, maybe ask how we can help. No policy issues.'}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 77, 'total_tokens': 134, 'completion_time': 0.13295972, 'completion_tokens_details': {'reasoning_tokens': 33}, 'prompt_time': 0.003087279, 'prompt_tokens_details': None, 'queue_time': 0.054536771, 'total_time': 0.136046999}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_70d048ba3c', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--0e5054c2-3aa2-41b1-a2bd-bedf11aed54a-0', usage_metadata={'input_tok

In [62]:
# Get full state history for a thread
# Returns all checkpoints in reverse chronological order

config = {"configurable": {"thread_id": "conversation_1"}}
history = list(simple_chatbot.get_state_history(config))

print(f"=== State History (Total checkpoints: {len(history)}) ===")

for i, checkpoint in enumerate(history[:3]):  # Show first 3
    print(f"\nCheckpoint {i+1}:")
    print(f"  Messages: {len(checkpoint.values.get('messages', []))}")
    print(f"  Next: {checkpoint.next}")
    print(f"  Created: {checkpoint.created_at}")

=== State History (Total checkpoints: 6) ===

Checkpoint 1:
  Messages: 4
  Next: ()
  Created: 2025-12-11T17:10:09.264529+00:00

Checkpoint 2:
  Messages: 3
  Next: ('chatbot',)
  Created: 2025-12-11T17:10:09.103672+00:00

Checkpoint 3:
  Messages: 2
  Next: ('__start__',)
  Created: 2025-12-11T17:10:09.102832+00:00


## Part 4: Human-in-the-Loop with Interrupts

**Use Cases**:
- Get approval before taking actions
- Collect additional information
- Review AI decisions
- Multi-step workflows with human oversight

**How it Works**:
1. Node calls `interrupt()` to pause execution
2. Graph saves current state and stops
3. Human reviews state and provides input
4. Graph resumes from exact same point with human input

In [63]:
# State for approval bot
# We'll build a bot that needs approval before sending emails

class ApprovalBotState(MessagesState):
    email_draft: str  # The drafted email
    approved: bool    # Whether human approved it
    recipient: str    # Who to send to

print("Approval bot state defined")

Approval bot state defined


In [64]:
# Nodes for approval workflow

def draft_email_node(state: ApprovalBotState) -> dict:
    """Generate email draft based on user request"""
    messages = state['messages']
    recipient = state.get('recipient', 'recipient@example.com')
    
    # Ask LLM to draft email
    prompt = f"Draft a professional email to {recipient} based on this request: {messages[-1].content}"
    response = llm.invoke([HumanMessage(content=prompt)])
    
    return {
        "email_draft": response.content,
        "messages": [AIMessage(content=f"I've drafted an email. Please review it.")]
    }

def approval_node(state: ApprovalBotState) -> Command[Literal["send_email", END]]:
    """
    Pause execution and wait for human approval.
    This is the key node that uses interrupt().
    """
    # Show the draft to human and wait for approval
    # interrupt() pauses execution here
    human_response = interrupt(
        {
            "draft": state['email_draft'],
            "recipient": state['recipient'],
            "question": "Do you approve this email? (yes/no)"
        }
    )
    
    # When resumed, human_response contains the human's input
    approved = human_response.get('approved', False)
    
    # Route based on approval
    if approved:
        return Command(
            update={"approved": True},
            goto="send_email"
        )
    else:
        return Command(
            update={
                "approved": False,
                "messages": [AIMessage(content="Email cancelled.")]
            },
            goto=END
        )

def send_email_node(state: ApprovalBotState) -> dict:
    """Simulate sending email"""
    # In production, this would actually send the email
    return {
        "messages": [AIMessage(content=f"Email sent to {state['recipient']}!")]
    }

print("Approval workflow nodes defined")

Approval workflow nodes defined


In [65]:
# Build approval bot graph

# Must use checkpointer for interrupts to work
approval_checkpointer = MemorySaver()

workflow = StateGraph(ApprovalBotState)
workflow.add_node("draft_email", draft_email_node)
workflow.add_node("approval", approval_node)
workflow.add_node("send_email", send_email_node)

workflow.add_edge(START, "draft_email")
workflow.add_edge("draft_email", "approval")
workflow.add_edge("send_email", END)

approval_bot = workflow.compile(checkpointer=approval_checkpointer)

print("Approval bot compiled")

Approval bot compiled


In [66]:
# Test the approval workflow
# Step 1: Start the workflow - it will pause at approval_node

config = {"configurable": {"thread_id": "email_1"}}

initial_result = approval_bot.invoke(
    {
        "messages": [HumanMessage(content="Send a thank you email for the meeting")],
        "recipient": "boss@company.com",
        "email_draft": "",
        "approved": False
    },
    config=config
)

print("=== Workflow Paused ===")
print("Graph stopped at interrupt. Check state:")

# Check what the interrupt is asking for
state = approval_bot.get_state(config)
print(f"\nInterrupt data: {state.tasks[0].interrupts}")

=== Workflow Paused ===
Graph stopped at interrupt. Check state:

Interrupt data: (Interrupt(value={'draft': '**Subject:** Thank You for Today’s Meeting  \n\n**To:** boss@company.com  \n\n---\n\nDear [Boss’s Name],\n\nI wanted to thank you for taking the time to meet with me today. I appreciate the opportunity to discuss [briefly mention the main topic or project, e.g., “the upcoming product launch strategy”] and to receive your valuable insights.\n\nYour guidance on [specific point discussed, e.g., “prioritizing the market research phase and aligning our timeline with the sales team’s objectives”] was especially helpful. I am confident that, with your direction, we can move forward effectively and achieve the desired outcomes.\n\nAs a next step, I will [outline any agreed‑upon actions, e.g., “prepare a detailed project plan and share it with the team by Friday”]. Please let me know if there are any additional items you’d like me to address.\n\nThank you again for your support and lead

In [67]:
# Step 2: Human reviews and approves
# Resume execution with Command.resume

# Simulate human approval
human_decision = Command(
    resume={"approved": True}  # Human approved the email
)

# Resume the graph
final_result = approval_bot.invoke(
    human_decision,
    config=config
)

print("=== Workflow Resumed ===")
print(f"Final message: {final_result['messages'][-1].content}")
print(f"Approved: {final_result['approved']}")

=== Workflow Resumed ===
Final message: Email sent to boss@company.com!
Approved: True


## Part 5: Cross-Thread Memory with Stores

**The Problem**:
- Checkpointers save state per thread (per conversation)
- What if we want to remember things ACROSS all conversations?
- Example: User preferences, profile info, learned facts

**The Solution - Stores**:
- Store data that persists across ALL threads
- Namespaced by user_id or other identifier
- Perfect for user profiles, preferences, long-term memory

In [68]:
# Create a store for cross-thread memory
# This will remember information across different conversations

memory_store = InMemoryStore()

print("Memory store created")

Memory store created


In [69]:
# State for chatbot with cross-thread memory

class MemoryBotState(MessagesState):
    user_id: str  # Identifies the user across threads

print("Memory bot state defined")

Memory bot state defined


In [70]:
# Nodes that use the store

def chatbot_with_memory(state: MemoryBotState, config: RunnableConfig, *, store: BaseStore) -> dict:
    """
    Chatbot that can access and update cross-thread memory.
    Notice the 'store' parameter - this gives us access to the memory store.
    """
    user_id = state['user_id']
    messages = state['messages']
    
    # Define namespace for this user's memories
    namespace = (user_id, "preferences")
    
    # Search for relevant memories
    # This searches across ALL conversations with this user
    memories = store.search(namespace)
    
    # Build context from memories
    memory_context = ""
    if memories:
        memory_context = "\n".join([
            f"- {mem.value.get('fact', '')}" 
            for mem in memories
        ])
    
    # Add system message with memory context
    system_content = "You are a helpful assistant."
    if memory_context:
        system_content += f"\n\nWhat you know about this user:\n{memory_context}"
    
    full_messages = [SystemMessage(content=system_content)] + messages
    
    # Generate response
    response = llm.invoke(full_messages)
    
    # Check if user shared new information to remember
    last_msg = messages[-1].content.lower()
    
    # Simple logic to detect preferences (in production, use an LLM)
    if "my favorite" in last_msg or "i like" in last_msg or "i prefer" in last_msg:
        # Store this as a new memory
        memory_id = str(uuid.uuid4())
        store.put(
            namespace,
            memory_id,
            {"fact": messages[-1].content}
        )
    
    return {"messages": [response]}

print("Memory-enabled node defined")

Memory-enabled node defined


In [71]:
# Build graph with BOTH checkpointer AND store

memory_checkpointer = MemorySaver()

workflow = StateGraph(MemoryBotState)
workflow.add_node("chatbot", chatbot_with_memory)
workflow.add_edge(START, "chatbot")
workflow.add_edge("chatbot", END)

# Compile with BOTH checkpointer (for thread state) and store (for cross-thread memory)
memory_bot = workflow.compile(
    checkpointer=memory_checkpointer,
    store=memory_store  # This is the key addition
)

print("Memory bot compiled with checkpointer and store")

Memory bot compiled with checkpointer and store


In [72]:
# Test 1: First conversation - share a preference

config = {
    "configurable": {
        "thread_id": "user_bob_conversation_1",
        "user_id": "user_bob"  # This identifies the user
    }
}

result1 = memory_bot.invoke(
    {
        "messages": [HumanMessage(content="My favorite color is blue")],
        "user_id": "user_bob"
    },
    config=config
)

print("=== First Conversation ===")
print(f"User: My favorite color is blue")
print(f"AI: {result1['messages'][-1].content}")

=== First Conversation ===
User: My favorite color is blue
AI: That’s a great choice! Blue is often associated with calmness, creativity, and depth. Do you have a particular shade of blue you love most?


In [73]:
# Test 2: DIFFERENT conversation (different thread) - should still remember

new_config = {
    "configurable": {
        "thread_id": "user_bob_conversation_2",  # Different thread!
        "user_id": "user_bob"  # Same user
    }
}

result2 = memory_bot.invoke(
    {
        "messages": [HumanMessage(content="What's my favorite color?")],
        "user_id": "user_bob"
    },
    config=new_config
)

print("\n=== Different Conversation (Should Remember) ===")
print(f"User: What's my favorite color?")
print(f"AI: {result2['messages'][-1].content}")


=== Different Conversation (Should Remember) ===
User: What's my favorite color?
AI: Your favorite color is blue.


In [74]:
# Inspect what's in the store

namespace = ("user_bob", "preferences")
all_memories = memory_store.search(namespace)

print("=== Stored Memories ===")
for mem in all_memories:
    print(f"\nMemory ID: {mem.key}")
    print(f"Fact: {mem.value['fact']}")
    print(f"Created: {mem.created_at}")

=== Stored Memories ===

Memory ID: b5cd655b-46f2-4c1d-8dfa-431718109cf4
Fact: My favorite color is blue
Created: 2025-12-11 17:10:10.895478+00:00

Memory ID: 0072b56b-1b28-44ba-94e7-519f7ebbe170
Fact: What's my favorite color?
Created: 2025-12-11 17:10:11.270532+00:00


## Part 6: Production-Ready Patterns

**Key Production Considerations**:
1. Use persistent storage (Postgres, SQLite) instead of in-memory
2. Implement proper error handling and retry logic
3. Add monitoring and logging
4. Handle concurrent access to threads
5. Implement cleanup for old threads

Let's build a production-ready chatbot pattern.

In [75]:
# Production state with metadata

class ProductionBotState(MessagesState):
    user_id: str
    session_id: str
    error_count: int  # Track errors for retry logic
    last_action: str  # Track what we last did

print("Production state defined")

Production state defined


In [76]:
# Production nodes with error handling

def production_chatbot_node(state: ProductionBotState, config: RunnableConfig) -> dict:
    """
    Production-ready chatbot with error handling.
    """
    try:
        messages = state['messages']
        
        # Add metadata to context
        metadata = config.get('metadata', {})
        session_info = f"Session: {state.get('session_id', 'unknown')}"
        
        # Generate response
        response = llm.invoke(messages)
        
        return {
            "messages": [response],
            "last_action": "chat_response",
            "error_count": 0  # Reset on success
        }
        
    except Exception as e:
        # Log error in production
        print(f"Error in chatbot: {e}")
        
        error_count = state.get('error_count', 0) + 1
        
        # If too many errors, give up
        if error_count >= 3:
            return {
                "messages": [AIMessage(content="I'm having technical difficulties. Please try again later.")],
                "last_action": "error_max_retries",
                "error_count": error_count
            }
        
        # Otherwise, increment error count and continue
        return {
            "messages": [AIMessage(content="I encountered an error. Let me try again.")],
            "last_action": "error_retry",
            "error_count": error_count
        }

print("Production node with error handling defined")

Production node with error handling defined


In [77]:
# Build production graph

prod_checkpointer = MemorySaver()

workflow = StateGraph(ProductionBotState)
workflow.add_node("chatbot", production_chatbot_node)
workflow.add_edge(START, "chatbot")
workflow.add_edge("chatbot", END)

production_bot = workflow.compile(checkpointer=prod_checkpointer)

print("Production bot compiled")

Production bot compiled


In [78]:
# Test production bot

config = {
    "configurable": {
        "thread_id": "prod_session_1",
        "user_id": "user_123"
    },
    "metadata": {
        "environment": "production",
        "version": "1.0"
    }
}

result = production_bot.invoke(
    {
        "messages": [HumanMessage(content="Hello")],
        "user_id": "user_123",
        "session_id": "prod_session_1",
        "error_count": 0,
        "last_action": "init"
    },
    config=config
)

print("=== Production Bot Test ===")
print(f"Response: {result['messages'][-1].content}")
print(f"Last action: {result['last_action']}")
print(f"Error count: {result['error_count']}")

=== Production Bot Test ===
Response: Hello! How can I assist you today?
Last action: chat_response
Error count: 0


## Part 7: Advanced State Management - Update State

Sometimes you need to manually modify the graph state.
This is useful for:
- Correcting mistakes
- Adding context mid-conversation
- Implementing human edits
- Branching conversations

In [79]:
# Get current state
config = {"configurable": {"thread_id": "conversation_1"}}
current_state = simple_chatbot.get_state(config)

print("=== Current State ===")
print(f"Messages: {len(current_state.values['messages'])}")

# Update state - add a system message
simple_chatbot.update_state(
    config,
    {
        "messages": [SystemMessage(content="Remember to be extra friendly")]
    }
)

# Check updated state
updated_state = simple_chatbot.get_state(config)
print(f"\nAfter update - Messages: {len(updated_state.values['messages'])}")
print(f"Latest message: {updated_state.values['messages'][-1].content}")

=== Current State ===
Messages: 4

After update - Messages: 5
Latest message: Remember to be extra friendly


## Part 8: Streaming with Persistence

Streaming works seamlessly with checkpointers.
Each streamed chunk is still checkpointed properly.

In [80]:
# Stream responses with persistence

config = {"configurable": {"thread_id": "stream_conversation"}}

print("=== Streaming with Persistence ===")

for chunk in simple_chatbot.stream(
    {
        "messages": [HumanMessage(content="Tell me about transformers in deep learning")],
        "user_name": "Student"
    },
    config=config,
    stream_mode="updates"  # Stream each node's output
):
    for node_name, node_output in chunk.items():
        print(f"\n[Node: {node_name}]")
        if 'messages' in node_output:
            print(f"Output: {node_output['messages'][-1].content[:100]}...")

# Verify state was saved
state = simple_chatbot.get_state(config)
print(f"\nState saved: {len(state.values['messages'])} messages")

=== Streaming with Persistence ===

[Node: chatbot]
Output: ## Transformers in Deep Learning – A High‑Level Overview  

Transformers have become the dominant ar...

State saved: 2 messages


## Complete Example

Putting it all together: A production chatbot with:
- Persistence (checkpointer)
- Cross-thread memory (store)
- Error handling
- Metadata tracking
- Proper state management

In [83]:
# Complete production-ready chatbot

class CompleteBotState(MessagesState):
    user_id: str
    session_id: str
    error_count: int

def complete_chatbot(state: CompleteBotState, config: RunnableConfig, *, store: BaseStore) -> dict:
    """Complete chatbot with all features"""
    try:
        user_id = state['user_id']
        messages = state['messages']
        
        # Load user preferences from store
        namespace = (user_id, "preferences")
        memories = store.search(namespace)
        
        # Build context
        memory_text = "\n".join([m.value.get('fact', '') for m in memories])
        system_msg = f"You are a helpful assistant.\n\nUser info:\n{memory_text}" if memory_text else "You are a helpful assistant."
        
        full_messages = [SystemMessage(content=system_msg)] + messages
        
        # Generate response
        response = llm.invoke(full_messages)
        
        # Extract and store new information if mentioned
        last_msg = messages[-1].content.lower()
        if any(keyword in last_msg for keyword in ["my favorite", "i like", "i prefer"]):
            store.put(namespace, str(uuid.uuid4()), {"fact": messages[-1].content})
        
        return {
            "messages": [response],
            "error_count": 0
        }
        
    except Exception as e:
        error_count = state.get('error_count', 0) + 1
        return {
            "messages": [AIMessage(content="I encountered an error. Please try again.")],
            "error_count": error_count
        }

# Build complete bot
complete_checkpointer = MemorySaver()
complete_store = InMemoryStore()

workflow = StateGraph(CompleteBotState)
workflow.add_node("chatbot", complete_chatbot)
workflow.add_edge(START, "chatbot")
workflow.add_edge("chatbot", END)

complete_bot = workflow.compile(
    checkpointer=complete_checkpointer,
    store=complete_store
)

print("Complete  bot ready")

Complete  bot ready


In [82]:
# Test complete bot

config = {
    "configurable": {
        "thread_id": "complete_test_1",
        "user_id": "alice"
    }
}

# First interaction - share preference
result1 = complete_bot.invoke(
    {
        "messages": [HumanMessage(content="My favorite programming language is Python")],
        "user_id": "alice",
        "session_id": "test_1",
        "error_count": 0
    },
    config=config
)

print("=== First Interaction ===")
print(f"AI: {result1['messages'][-1].content}")

# New thread, same user - should remember
new_config = {
    "configurable": {
        "thread_id": "complete_test_2",
        "user_id": "alice"  # Same user
    }
}

result2 = complete_bot.invoke(
    {
        "messages": [HumanMessage(content="What programming language do I prefer?")],
        "user_id": "alice",
        "session_id": "test_2",
        "error_count": 0
    },
    config=new_config
)

print("\n=== New Thread (Should Remember) ===")
print(f"AI: {result2['messages'][-1].content}")

=== First Interaction ===
AI: That's awesome! Python is a fantastic language—readable, versatile, and backed by a huge community. What do you enjoy most about it? Do you have a favorite library or project you’ve been working on?

=== New Thread (Should Remember) ===
AI: You mentioned that your favorite programming language is **Python**.
